In [91]:
import hashlib
import hmac
import json
import time
import requests
import os
import pandas as pd
import numpy as np
from datetime import datetime

def gen_sign(api_secret, payload_string=None):
    return hmac.new(api_secret.encode('utf-8'), payload_string.encode('utf-8'), hashlib.sha256).hexdigest()

def gen_query_param(url, query_param):
    req = requests.PreparedRequest()
    req.prepare_url(url, query_param)
    return req.url.replace(url,"")

In [97]:
str(round(int(datetime.strptime("13-11-2024", "%d-%m-%Y").timestamp()) * 1000))

'1731430800000'

In [104]:
# call api for my-order-history

def call_my_order_history_each_coin(sym):
    
    host = 'https://api.bitkub.com'
    path = '/api/v3/market/my-order-history'
    api_key = os.environ['BITKUB_API_KEY']
    api_secret = os.environ['BITKUB_API_SECRET']

    ts = str(round(time.time() * 1000))
    param = {
        'sym':sym, # symbol in quote_base format: e.g. btc_thb
        'start': str(round(int(datetime.strptime("13-11-2024", "%d-%m-%Y").timestamp()) * 1000)),
        'lmt': 1000
    }
    query_param = gen_query_param(host+path, param)

    payload = []
    payload.append(ts)
    payload.append('GET')
    payload.append(path)
    payload.append(query_param)

    sig = gen_sign(api_secret, ''.join(payload))
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'X-BTK-TIMESTAMP': ts,
        'X-BTK-SIGN': sig,
        'X-BTK-APIKEY': api_key
    }

    response_my_order_history = requests.request('GET', f'{host}{path}{query_param}', headers=headers, data={}, verify=True)
    return response_my_order_history

In [105]:
# call api for all symbols

def call_all_symbols():

    host = 'https://api.bitkub.com'
    path = '/api/market/symbols'

    ts = str(round(time.time() * 1000))
    param = {
    }
    query_param = gen_query_param(host+path, param)

    payload = []
    payload.append(ts)
    payload.append('GET')
    payload.append(path)
    payload.append(query_param)

    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'X-BTK-TIMESTAMP': ts,
    }

    response_all_symbols = requests.request('GET', f'{host}{path}{query_param}', headers=headers, data={}, verify=True)
    return response_all_symbols

In [106]:
# convert to df for all symbols
data = call_all_symbols().json()
result = data['result']
df_all_symbols = pd.DataFrame(result)
df_all_symbols['symbol'] = df_all_symbols['symbol'].str.lower().str.split('_').str[::-1].str.join('_')
df_all_symbols['symbol']

0       btc_thb
1       eth_thb
2       ada_thb
3       bch_thb
4      usdt_thb
         ...   
156    tnsr_thb
157     dbr_thb
158    kaia_thb
159    carv_thb
160     fet_thb
Name: symbol, Length: 161, dtype: object

In [113]:
# convert to df for my-order-history

df_my_order_history_all = pd.DataFrame()

for symbol in df_all_symbols['symbol']:
    
    sym = symbol
    data = call_my_order_history_each_coin(sym).json()
    result = data['result']

    if result == [] or result == None:
        continue

    print(sym)
    df_my_order_history = pd.DataFrame(result)
    df_my_order_history['ts'] = pd.to_datetime(df_my_order_history['ts'], unit='ms')
    df_my_order_history_all = pd.concat([df_my_order_history_all, df_my_order_history], ignore_index=True)

df_my_order_history_all.head()

btc_thb
ada_thb
bch_thb
xrp_thb
ctxc_thb
xlm_thb
six_thb
doge_thb
ksm_thb
velo_thb


,txn_id,order_id,hash,parent_order_id,parent_order_hash,super_order_id,super_order_hash,client_id,taken_by_me,is_maker,side,type,rate,fee,credit,amount,ts
0,BTCSELL0024178421,298473336,E5a9FBF5rPEWA7U1VncukDmqSmPumu,0,JiWPghZbgX9MjqZ9H2K,0,JiWPghZbgX9MjqZ9H2K,,False,False,sell,limit,3267709.04,22.15,0,0.00271022,2024-11-26 05:22:37.898
1,BTCBUY0024104677,357906601,3xs6wsMQkpBK1vtRR261Lv2KtdomS,0,51o5gooMYcNqCWwA32,0,51o5gooMYcNqCWwA32,,False,False,buy,limit,3388999.95,23.02,0,9207.98,2024-11-22 01:54:05.409
2,BTCBUY0024060282,297277680,E5a9FBF5rPEWA7SuX6GcRNpieStUTu,0,JiWPghZbgX9MjqZ9H2K,0,JiWPghZbgX9MjqZ9H2K,,False,True,sell,limit,3210000,46.69,0,0.00581791,2024-11-20 07:40:20.438
3,BTCSELL0024059027,357335554,3xs6wsMQkpBK1vtR5WKpQU5vSHuQc,0,51o5gooMYcNqCWwA32,0,51o5gooMYcNqCWwA32,,False,True,buy,limit,3186000,46.46,0,18582.35,2024-11-20 05:02:45.788
4,BTCBUY0024034030,296826830,E5a9FBF5rPEWA7RqXB4gowQnQLXs2B,296825870,E5a9FBF5rPEWA7RqXB3wza19EdZs9M,296825807,E5a9FBF5rPEWA7RqXB3wzJgGKVKtmR,,False,True,sell,limit,3162000,1.08,0,0.00013607,2024-11-18 11:34:28.226


In [114]:
# transform data amount_thb
df_my_order_history_all['rate'] = pd.to_numeric(df_my_order_history_all['rate'])
df_my_order_history_all['amount'] = pd.to_numeric(df_my_order_history_all['amount'])
df_my_order_history_all['amount_thb'] = np.where(df_my_order_history_all['side'] == 'sell', df_my_order_history_all['amount'] * df_my_order_history_all['rate'], df_my_order_history_all['amount'])
df_my_order_history_all['amount_thb'] = np.round(df_my_order_history_all['amount_thb'], 2)

# transform data ticker
df_my_order_history_all['ticker'] = df_my_order_history_all['txn_id'].str.split('(BUY|SELL)').str[0]
df_my_order_history_all.head()

,txn_id,order_id,hash,parent_order_id,parent_order_hash,super_order_id,super_order_hash,client_id,taken_by_me,is_maker,side,type,rate,fee,credit,amount,ts,amount_thb,ticker
0,BTCSELL0024178421,298473336,E5a9FBF5rPEWA7U1VncukDmqSmPumu,0,JiWPghZbgX9MjqZ9H2K,0,JiWPghZbgX9MjqZ9H2K,,False,False,sell,limit,3267709.04,22.15,0,0.002710,2024-11-26 05:22:37.898,8856.21,BTC
1,BTCBUY0024104677,357906601,3xs6wsMQkpBK1vtRR261Lv2KtdomS,0,51o5gooMYcNqCWwA32,0,51o5gooMYcNqCWwA32,,False,False,buy,limit,3388999.95,23.02,0,9207.980000,2024-11-22 01:54:05.409,9207.98,BTC
2,BTCBUY0024060282,297277680,E5a9FBF5rPEWA7SuX6GcRNpieStUTu,0,JiWPghZbgX9MjqZ9H2K,0,JiWPghZbgX9MjqZ9H2K,,False,True,sell,limit,3210000.00,46.69,0,0.005818,2024-11-20 07:40:20.438,18675.49,BTC
3,BTCSELL0024059027,357335554,3xs6wsMQkpBK1vtR5WKpQU5vSHuQc,0,51o5gooMYcNqCWwA32,0,51o5gooMYcNqCWwA32,,False,True,buy,limit,3186000.00,46.46,0,18582.350000,2024-11-20 05:02:45.788,18582.35,BTC
4,BTCBUY0024034030,296826830,E5a9FBF5rPEWA7RqXB4gowQnQLXs2B,296825870,E5a9FBF5rPEWA7RqXB3wza19EdZs9M,296825807,E5a9FBF5rPEWA7RqXB3wzJgGKVKtmR,,False,True,sell,limit,3162000.00,1.08,0,0.000136,2024-11-18 11:34:28.226,430.25,BTC


In [115]:
df_my_order_history_all = df_my_order_history_all.sort_values(by=['ts'], ascending=False)
df_my_order_history_all

,txn_id,order_id,hash,parent_order_id,parent_order_hash,super_order_id,super_order_hash,client_id,taken_by_me,is_maker,side,type,rate,fee,credit,amount,ts,amount_thb,ticker
18,BCHSELL0002199506,110358127,E5a9FBF5s13W5yWJ2X3eniJZc7UeYT,110358126,E5a9FBF5s13W5yWJ2X3eniJ4iwQA1q,110358126,E5a9FBF5s13W5yWJ2X3eniJ4iwQA1q,,False,False,sell,market,1.718400e+04,17.88,0,0.416088,2024-11-27 09:31:58.975,7150.05,BCH
19,BCHSELL0002199505,110358126,E5a9FBF5s13W5yWJ2X3eniJ4iwQA1q,0,JiWPghZbhMeiosecKMu,0,JiWPghZbhMeiosecKMu,,False,False,sell,market,1.718400e+04,4.85,0,0.112744,2024-11-27 09:31:58.975,1937.40,BCH
41,DOGESELL0020290364,234652805,E5a9FBF7ViN85YGm5KWy37uH6FP19D,0,JiWPghZdueWeV1hvhZd,0,JiWPghZdueWeV1hvhZd,,False,False,sell,market,1.353000e+01,23.45,0,693.144444,2024-11-27 09:31:34.598,9378.24,DOGE
42,DOGESELL0020288256,281466360,3xs6wsMR8QsYxx92iG4BheT1PPXQU,0,51o5gooN3sBo6tieTA,0,51o5gooN3sBo6tieTA,,False,True,buy,limit,1.350000e+01,23.46,0,9380.910000,2024-11-27 05:55:17.288,9380.91,DOGE
30,CTXCSELL0002373286,39440190,3xs6wsMR1LLdmSDdy2b5irHgQcQro,39440189,3xs6wsMR1LLdmSDdy2b5gjHkVUvTV,39440189,3xs6wsMR1LLdmSDdy2b5gjHkVUvTV,,False,False,sell,limit,2.002000e+01,20.15,0,402.572762,2024-11-27 05:48:13.298,8059.51,CTXC
31,CTXCSELL0002373285,39440189,3xs6wsMR1LLdmSDdy2b5gjHkVUvTV,0,JiWPghZdBL6TxnY8rsm,0,JiWPghZdBL6TxnY8rsm,,False,False,sell,limit,2.002000e+01,3.37,0,67.179820,2024-11-27 05:48:13.298,1344.94,CTXC
20,BCHBUY0002199265,137040126,3xs6wsMQkxHszojtw6LFUi8nugzCY,0,51o5gooMYoNEYv4joW,0,51o5gooMYoNEYv4joW,,False,False,buy,market,1.719716e+04,22.8,0,9117.210000,2024-11-27 03:10:36.825,9117.21,BCH
32,CTXCBUY0002371849,41709611,fwQ6dnQZhdczM6AihvBbv1TFb3E,0,51o5gooMtH7DuWhneL,0,51o5gooMtH7DuWhneL,,False,False,buy,limit,1.936000e+01,22.8,0,9117.210000,2024-11-27 02:55:28.058,9117.21,CTXC
51,VELOBUY0000614273,3973350,fwQ6dnXsCjJ1zRX1bhNduBNqjUw,3973327,fwQ6dnXsCjJ1zRX1bhG6wk9uRib,3973327,fwQ6dnXsCjJ1zRX1bhG6wk9uRib,,False,True,sell,limit,4.300000e+00,11.19,0,1040.021265,2024-11-26 17:03:28.456,4472.09,VELO
52,VELOBUY0000614271,3973327,fwQ6dnXsCjJ1zRX1bhG6wk9uRib,0,JiWPghcwrqb9rV7drCj,0,JiWPghcwrqb9rV7drCj,,False,True,sell,limit,4.300000e+00,34.53,0,3211.176744,2024-11-26 17:03:03.448,13808.06,VELO
